# __Cargar datos__

In [18]:
import joblib

dir_list = [
    {
    "dir_name": "NYU",
    "single_phenotype_file": True,
    },
    {
    "dir_name": "NeuroIMAGE",
    "single_phenotype_file": True,
    },
    {
    "dir_name": "KKI",
    "single_phenotype_file": True,
    },
    {
    "dir_name": "OHSU",
    "single_phenotype_file": True,
    },
    {
    "dir_name": "Peking",
    "single_phenotype_file": False,
    },
    ]


n_rois=18

all_data = {}
site2idx = {}         # Diccionario para codificar los sitios como enteros
train_ts_list = []
train_labels = []
train_sites = []      # <--- Lista paralela con los sitios

for idx, dataset in enumerate(dir_list):
    dir_name = dataset["dir_name"]
    site2idx[dir_name] = idx          # Asignamos un entero a cada sitio

    data = joblib.load(f"../raw-bold-data/{n_rois}-rois-dataset/{dir_name}.pkl")
    ts_list, labels = data["data"], data["labels"]
    all_data[dir_name] = {"data": ts_list, "labels": labels}

    train_ts_list.extend(ts_list)
    train_labels.extend(labels)
    train_sites.extend([idx] * len(ts_list))  # Añadir código del sitio para cada muestra

    print("Dataset: {}, Total: {}, TDAH: {}, Control: {}".format(dir_name, len(ts_list), sum(labels),
                                                               len(ts_list) - sum(labels)))
    print(10 * "*")

idx2site = {v: k for k, v in site2idx.items()}

# Estadísticas
print("\nTotal de muestras:", len(train_ts_list))
print("Total con TDHA:", sum(train_labels))
print("Número de sitios:", len(site2idx))

Dataset: NYU, Total: 177, TDAH: 90, Control: 87
**********
Dataset: NeuroIMAGE, Total: 39, TDAH: 17, Control: 22
**********
Dataset: KKI, Total: 78, TDAH: 20, Control: 58
**********
Dataset: OHSU, Total: 66, TDAH: 28, Control: 38
**********
Dataset: Peking, Total: 183, TDAH: 74, Control: 109
**********

Total de muestras: 543
Total con TDHA: 229
Número de sitios: 5


## __FC GENERATOR__

In [73]:
import numpy as np

class DynamicFcBuilder:
    def __init__(self, win_len=40, step=15):
        self.win_len = win_len
        self.step = step

    def sliding_windows(self, ts):
        ts = np.ascontiguousarray(ts)  # mejora acceso a memoria
        T = ts.shape[1]
        starts = range(0, T - self.win_len + 1, self.step)
        return np.stack([ts[:, s:s+self.win_len] for s in starts], axis=0)

    def ts_to_fc(self, windows):
        fcs = []
        for w in windows:
            # Normalización robusta
            mu = w.mean(axis=1, keepdims=True)
            std = w.std(axis=1, keepdims=True)
            std[std == 0.0] = 1.0
            w_norm = (w - mu) / std

            # Correlación rápida con z-transformación
            r = np.dot(w_norm, w_norm.T) / w.shape[1]
            np.fill_diagonal(r, 0.0)
            z = np.arctanh(np.clip(r, -0.999, 0.999))
            fcs.append(z.astype(np.float32))

        return np.stack(fcs)

    def compute_dynamic_fc(self, ts):
        return self.ts_to_fc(self.sliding_windows(ts))


In [74]:
fc_builder = DynamicFcBuilder(win_len=40, step=15)

resumen_data = []

for site_name, site_data in all_data.items():
    ts_list = site_data["data"]
    matrices_por_sujeto = [fc_builder.compute_dynamic_fc(ts).shape[0] for ts in ts_list]
    total_fc = sum(matrices_por_sujeto)
    promedio_fc = np.mean(matrices_por_sujeto)

    resumen_data.append({
        "Sitio": site_name,
        "Sujetos": len(ts_list),
        "Matrices FC por sujeto": int(promedio_fc),
        "Matrices FC por sitio": total_fc,

    })

import pandas as pd
df_resumen = pd.DataFrame(resumen_data)
print(df_resumen)


        Sitio  Sujetos  Matrices FC por sujeto  Matrices FC por sitio
0         NYU      177                       9                   1593
1  NeuroIMAGE       39                      15                    585
2         KKI       78                       6                    522
3        OHSU       66                       3                    198
4      Peking      183                      13                   2379


#  __FCM-RAW-BOLD-GENERATOR__

In [19]:
import os
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm

# === Tu clase, tal cual ===
import numpy as np

class DynamicFcBuilder:
    def __init__(self, win_len=40, step=15):
        self.win_len = win_len
        self.step = step

    def sliding_windows(self, ts):
        ts = np.ascontiguousarray(ts)  # mejora acceso a memoria
        T = ts.shape[1]
        starts = range(0, T - self.win_len + 1, self.step)
        return np.stack([ts[:, s:s+self.win_len] for s in starts], axis=0)

    def ts_to_fc(self, windows):
        fcs = []
        for w in windows:
            # Normalización robusta
            mu = w.mean(axis=1, keepdims=True)
            std = w.std(axis=1, keepdims=True)
            std[std == 0.0] = 1.0
            w_norm = (w - mu) / std

            # Correlación rápida con z-transformación
            r = np.dot(w_norm, w_norm.T) / w.shape[1]
            np.fill_diagonal(r, 0.0)
            z = np.arctanh(np.clip(r, -0.999, 0.999))
            fcs.append(z.astype(np.float32))

        return np.stack(fcs)

    def compute_dynamic_fc(self, ts):
        return self.ts_to_fc(self.sliding_windows(ts))


# ====== Parámetros ======
n_rois   = 18
win_len  = 40
step     = 15
fc_builder = DynamicFcBuilder(win_len=win_len, step=step)

# Directorio de salida
OUT_DIR = "18-fcm"
os.makedirs(OUT_DIR, exist_ok=True)

# all_data debe existir (como lo creaste antes):
# all_data = {"NYU": {"data": [...], "labels": [...]}, ...}

resumen_data = []

def process_site(site_name: str, site_data: dict):
    """
    Genera dFC por sujeto (W_i, 116, 116) y guarda {site_name}_dfc.pkl.
    Estructura de guardado:
      {
        "site": str,
        "win_len": int,
        "step": int,
        "n_rois": int,
        "data":   [np.ndarray (W_i, 116, 116), ...],
        "labels": [int, ...]
      }
    Retorna: (num_sujetos, matrices_por_sujeto_list)
    """
    ts_list   = site_data["data"]
    labels    = site_data["labels"]

    out_obj = {
        "site": site_name,
        "win_len": win_len,
        "step": step,
        "n_rois": n_rois,
        "data": [],
        "labels": []
    }

    matrices_por_sujeto = []
    skipped = 0

    for ts, y in tqdm(zip(ts_list, labels), total=len(ts_list), desc=f"{site_name}"):
        # Validaciones rápidas
        if ts.shape[0] != n_rois:
            raise ValueError(f"{site_name}: ROI mismatch. Esperado {n_rois}, recibido {ts.shape[0]}")
        T = ts.shape[1]
        if T < win_len:
            skipped += 1
            continue

        # dFC
        dfc = fc_builder.compute_dynamic_fc(ts)  # (W_i, 116, 116)
        if not np.isfinite(dfc).all():
            skipped += 1
            continue

        out_obj["data"].append(dfc)
        out_obj["labels"].append(int(y))
        matrices_por_sujeto.append(dfc.shape[0])

    # Guardar por sitio
    out_path = os.path.join(OUT_DIR, f"{site_name}_dfc.pkl")
    joblib.dump(out_obj, out_path, compress=3)

    print(f"[{site_name}] sujetos: {len(ts_list)} | guardados: {len(out_obj['data'])} | saltados: {skipped}")
    print(f"→ {out_path}")

    return len(out_obj["data"]), matrices_por_sujeto


# ====== Ejecutar por cada sitio y armar resumen ======
for site_name, site_data in all_data.items():
    sujetos_guardados, mats_ps = process_site(site_name, site_data)

    if len(mats_ps) == 0:
        promedio_fc = 0
        total_fc = 0
    else:
        promedio_fc = int(np.round(np.mean(mats_ps)))
        total_fc = int(np.sum(mats_ps))

    resumen_data.append({
        "Sitio": site_name,
        "Sujetos": sujetos_guardados,
        "Matrices FC por sujeto": promedio_fc,
        "Matrices FC por sitio": total_fc,
    })

df_resumen = pd.DataFrame(resumen_data)
print("\nResumen por sitio:")
print(df_resumen)

# (Opcional) guardar el resumen a CSV
df_resumen.to_csv(os.path.join(OUT_DIR, "resumen_dfc_por_sitio.csv"), index=False)


NYU: 100%|██████████| 177/177 [00:00<00:00, 3071.01it/s]


[NYU] sujetos: 177 | guardados: 177 | saltados: 0
→ 18-fcm/NYU_dfc.pkl


NeuroIMAGE: 100%|██████████| 39/39 [00:00<00:00, 2390.62it/s]


[NeuroIMAGE] sujetos: 39 | guardados: 39 | saltados: 0
→ 18-fcm/NeuroIMAGE_dfc.pkl


KKI: 100%|██████████| 78/78 [00:00<00:00, 5166.30it/s]


[KKI] sujetos: 78 | guardados: 78 | saltados: 0
→ 18-fcm/KKI_dfc.pkl


OHSU: 100%|██████████| 66/66 [00:00<00:00, 4532.08it/s]


[OHSU] sujetos: 66 | guardados: 66 | saltados: 0
→ 18-fcm/OHSU_dfc.pkl


Peking: 100%|██████████| 183/183 [00:00<00:00, 3070.21it/s]


[Peking] sujetos: 183 | guardados: 183 | saltados: 0
→ 18-fcm/Peking_dfc.pkl

Resumen por sitio:
        Sitio  Sujetos  Matrices FC por sujeto  Matrices FC por sitio
0         NYU      177                       9                   1593
1  NeuroIMAGE       39                      15                    585
2         KKI       78                       7                    522
3        OHSU       66                       3                    198
4      Peking      183                      13                   2379


# __FCM-AUGMENTED-BOLD__

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm

# ========= Clases (tal cual las pasaste) =========
class BoldAugmentor:
    def __init__(self, noise_std=0.02, shift_max=5,
                 flip_prob=0.1, drop_prob=0.1, scale_range=(0.95, 1.05)):
        self.noise_std = noise_std
        self.shift_max = shift_max
        self.flip_prob = flip_prob
        self.drop_prob = drop_prob
        self.scale_range = scale_range

    def augment(self, ts):
        ts = np.ascontiguousarray(ts)

        rand = np.random.rand
        shape = ts.shape

        if rand() < 0.5:
            ts += np.random.normal(0.0, self.noise_std, size=shape)

        if rand() < 0.3:
            shift = np.random.randint(-self.shift_max, self.shift_max + 1)
            if shift != 0:
                if shift > 0:
                    ts = np.pad(ts, ((0, 0), (shift, 0)), mode='constant')[:, :-shift]
                else:
                    ts = np.pad(ts, ((0, 0), (0, -shift)), mode='constant')[:, -shift:]

        if rand() < self.flip_prob:
            ts = ts[:, ::-1].copy()

        if rand() < self.drop_prob:
            drop_idx = np.random.randint(shape[0])
            ts[drop_idx] = 0.0

        if rand() < self.drop_prob:
            ts *= np.random.uniform(*self.scale_range, size=(shape[0], 1))

        return np.nan_to_num(ts, nan=0.0, posinf=1.0, neginf=-1.0)


class DynamicFcBuilder:
    def __init__(self, win_len=40, step=15):
        self.win_len = win_len
        self.step = step

    def sliding_windows(self, ts):
        ts = np.ascontiguousarray(ts)
        T = ts.shape[1]
        starts = range(0, T - self.win_len + 1, self.step)
        return np.stack([ts[:, s:s+self.win_len] for s in starts], axis=0)

    def ts_to_fc(self, windows):
        fcs = []
        for w in windows:
            mu = w.mean(axis=1, keepdims=True)
            std = w.std(axis=1, keepdims=True)
            std[std == 0.0] = 1.0
            w_norm = (w - mu) / std

            r = (w_norm @ w_norm.T) / w.shape[1]
            np.fill_diagonal(r, 0.0)
            z = np.arctanh(np.clip(r, -0.999, 0.999))
            fcs.append(z.astype(np.float32))
        return np.stack(fcs)

    def compute_dynamic_fc(self, ts):
        return self.ts_to_fc(self.sliding_windows(ts))


# ========= Parámetros =========
n_rois        = 116
win_len       = 40
step          = 15
AUG_REPS      = 2         # cuántas réplicas aumentadas por sujeto
INCLUDE_ORIG  = True      # incluir también la serie original
RANDOM_SEED   = 42        # para reproducibilidad

augmentor = BoldAugmentor(
    noise_std=0.02,
    shift_max=5,
    flip_prob=0.1,
    drop_prob=0.1,
    scale_range=(0.95, 1.05),
)
fc_builder = DynamicFcBuilder(win_len=win_len, step=step)

np.random.seed(RANDOM_SEED)

OUT_DIR = "../processed-dfc-aug"   # salida
os.makedirs(OUT_DIR, exist_ok=True)

# all_data debe existir: {"NYU":{"data":[(116,T),...],"labels":[...]}, ...}

def windows_count(T, win_len, step):
    return 0 if T < win_len else 1 + (T - win_len) // step


def process_site_with_augmentation(site_name: str, site_data: dict):
    """
    Genera (original + AUG_REPS aumentadas) por sujeto, calcula dFC y guarda {site}_dfc_aug.pkl
    Estructura:
      {
        "site": str,
        "win_len": int, "step": int, "n_rois": int,
        "include_original": bool, "aug_reps": int, "random_seed": int,
        "data":   [ (W_i, 116, 116), ... ],
        "labels": [ int, ... ],
        "augmented": [ bool, ... ],        # True si es réplica aumentada
        "source_index": [ int, ... ]       # índice del sujeto original
      }
    """
    ts_list = site_data["data"]
    labels  = site_data["labels"]

    out_obj = {
        "site": site_name,
        "win_len": win_len,
        "step": step,
        "n_rois": n_rois,
        "include_original": INCLUDE_ORIG,
        "aug_reps": AUG_REPS,
        "random_seed": RANDOM_SEED,
        "data": [],
        "labels": [],
        "augmented": [],
        "source_index": [],
    }

    mats_por_sample = []  # W_i por cada muestra (orig o aug)
    skipped = 0

    for subj_idx, (ts, y) in enumerate(tqdm(zip(ts_list, labels), total=len(ts_list), desc=site_name)):
        # validaciones base
        if ts.shape[0] != n_rois:
            raise ValueError(f"{site_name}: ROI mismatch. Esperado {n_rois}, recibido {ts.shape[0]}")
        T = ts.shape[1]
        if T < win_len:
            skipped += 1
            continue

        # Colección de series a convertir a dFC: original (opcional) + réplicas aumentadas
        series_list = []
        if INCLUDE_ORIG:
            series_list.append((ts, False))  # (serie, es_aug)

        for _ in range(AUG_REPS):
            ts_aug = augmentor.augment(ts.copy())
            series_list.append((ts_aug, True))

        # Convertir cada serie a dFC y guardar
        for ser, is_aug in series_list:
            dfc = fc_builder.compute_dynamic_fc(ser)  # (W_i, 116, 116)
            if not np.isfinite(dfc).all():
                # si algo rompió la numerica, salta esta entrada
                continue

            out_obj["data"].append(dfc)
            out_obj["labels"].append(int(y))
            out_obj["augmented"].append(bool(is_aug))
            out_obj["source_index"].append(int(subj_idx))
            mats_por_sample.append(dfc.shape[0])

    out_path = os.path.join(OUT_DIR, f"{site_name}_dfc_aug.pkl")
    joblib.dump(out_obj, out_path, compress=3)

    print(f"[{site_name}] sujetos originales: {len(ts_list)} | muestras guardadas (orig+aug): {len(out_obj['data'])} | sujetos saltados: {skipped}")
    print(f"→ {out_path}")

    return len(ts_list), len(out_obj["data"]), mats_por_sample


# ====== Ejecutar por sitio y resumen ======
resumen_rows = []
for site_name, site_data in all_data.items():
    n_orig, n_muestras, mats_list = process_site_with_augmentation(site_name, site_data)

    if len(mats_list) == 0:
        prom_w = 0
        total_w = 0
    else:
        prom_w = int(np.round(np.mean(mats_list)))  # ventanas promedio por muestra (orig+aug)
        total_w = int(np.sum(mats_list))            # total de matrices dFC (todas las muestras)

    resumen_rows.append({
        "Sitio": site_name,
        "Sujetos originales": n_orig,
        "Muestras (orig+aug)": n_muestras,
        "Matrices FC por muestra": prom_w,
        "Matrices FC por sitio": total_w,
    })

df_resumen = pd.DataFrame(resumen_rows)
print("\nResumen por sitio (con aumentación):")
print(df_resumen)

# (opcional) CSV de resumen
df_resumen.to_csv(os.path.join(OUT_DIR, "resumen_dfc_aug_por_sitio.csv"), index=False)


## __Channel Builder__

In [9]:
import numpy as np
from scipy.linalg import expm, eigh
from scipy.sparse.csgraph import laplacian
from scipy.stats import entropy
import networkx as nx
from joblib import Parallel, delayed

class FcChannelBuilder:
    def __init__(self,
                 use_mean=True,
                 use_abs=True,
                 use_std=True,
                 use_max=True,
                 use_min=True,
                 use_diffusion=False,
                 use_spec=False,
                 use_clustering=False,
                 use_entropy=False,
                 use_ddt=False,
                 eig_k=4,
                 ddt_scale=1.0,
                 t_list=None,
                 n_jobs=0,
                 verbose=False):
        # Canales base
        self.use_mean = use_mean
        self.use_abs = use_abs
        self.use_std = use_std
        self.use_max = use_max
        self.use_min = use_min

        # Canales agregados
        self.use_diffusion = use_diffusion
        self.use_spec = use_spec
        self.use_clustering = use_clustering
        self.use_entropy = use_entropy
        self.use_ddt = use_ddt

        # Parámetros adicionales
        self.eig_k = eig_k
        self.ddt_scale = ddt_scale
        self.t_list = [1.0] if t_list is None else t_list
        self.n_jobs = n_jobs
        self.verbose = verbose

    def build(self, fcs: np.ndarray) -> np.ndarray:
        """
        fcs: np.ndarray of shape (W, N, N), Fisher-z transformed
        Returns: np.ndarray of shape (C, N, N)
        """
        if not isinstance(fcs, np.ndarray) or fcs.ndim != 3:
            raise ValueError("❌ fcs debe ser un arreglo NumPy de shape (W, N, N)")

        chans = []

        if self.use_mean:
            chan = fcs.mean(axis=0)
            chans.append(chan)
            if self.verbose: print("[mean] shape:", chan.shape)

        if self.use_abs:
            chan = np.mean(np.abs(fcs), axis=0)
            chans.append(chan)
            if self.verbose: print("[abs] shape:", chan.shape)

        if self.use_std:
            chan = fcs.std(axis=0)
            chans.append(chan)
            if self.verbose: print("[std] shape:", chan.shape)

        if self.use_max:
            chan = fcs.max(axis=0)
            chans.append(chan)
            if self.verbose: print("[max] shape:", chan.shape)

        if self.use_min:
            chan = fcs.min(axis=0)
            chans.append(chan)
            if self.verbose: print("[min] shape:", chan.shape)

        if self.use_diffusion:
            chans.append(self._diffusion_channels(fcs))

        if self.use_spec:
            chans.append(self._laplacian_eig_channel(fcs, k=self.eig_k))

        if self.use_clustering:
            chans.append(self._clustering_channel(fcs))

        if self.use_entropy:
            chans.append(self._entropy_channel(fcs))

        if self.use_ddt:
            chans.append(self._ddt_channel(fcs))

        return np.stack(chans, axis=0).astype(np.float32, copy=False)

    # -------------------------------
    # Métodos auxiliares
    # -------------------------------

    def _diffusion_single(self, z, t):
        adj = np.abs(z)
        L = laplacian(adj, normed=True)
        return expm(-t * L)

    def _diffusion_channels(self, fcs):
        results = []
        for t in self.t_list:
            diffs = Parallel(n_jobs=self.n_jobs)(
                delayed(self._diffusion_single)(z, t) for z in fcs
            )
            chan_t = np.mean(diffs, axis=0, keepdims=True)
            results.append(chan_t)
        return np.concatenate(results, axis=0)  # shape: (len(t_list), N, N)

    def _laplacian_eig_channel(self, fcs, k=4):
        def eig_spec(z):
            adj = np.abs(z)
            L = laplacian(adj, normed=True)
            _, eigvecs = eigh(L)
            topk = eigvecs[:, :k]
            return topk @ topk.T  # (N, N)

        ev_matrices = Parallel(n_jobs=self.n_jobs)(
            delayed(eig_spec)(z) for z in fcs
        )
        return np.mean(ev_matrices, axis=0, keepdims=True)  # shape: (1, N, N)

    def _clustering_channel(self, fcs):
        def cluster_diag(z):
            G = nx.from_numpy_array(np.abs(z))
            clust = nx.clustering(G)
            vec = np.array([clust[i] for i in range(z.shape[0])])
            return np.diag(vec)

        mats = Parallel(n_jobs=self.n_jobs)(
            delayed(cluster_diag)(z) for z in fcs
        )
        return np.mean(mats, axis=0, keepdims=True)

    def _entropy_channel(self, fcs):
        def entropy_diag(z):
            abs_z = np.abs(z)
            norm_z = abs_z / (abs_z.sum(axis=1, keepdims=True) + 1e-6)
            ent = entropy(norm_z.T)
            return np.diag(ent)

        mats = Parallel(n_jobs=self.n_jobs)(
            delayed(entropy_diag)(z) for z in fcs
        )
        return np.mean(mats, axis=0, keepdims=True)

    def _ddt_channel(self, fcs):
        def ddt_transform(z):
            adj = np.abs(z)
            L = laplacian(adj, normed=True)
            return expm(-self.ddt_scale * L)

        mats = Parallel(n_jobs=self.n_jobs)(
            delayed(ddt_transform)(z) for z in fcs
        )
        return np.mean(mats, axis=0, keepdims=True)


## __Example__

In [10]:
import numpy as np
from tqdm import tqdm

# 1. Instancias
augmentor = BoldAugmentor()
fc_builder = DynamicFcBuilder(win_len=40, step=15)
channel_builder = FcChannelBuilder(
    use_mean=True,         # Debe estar en True
    use_abs=True,          # También en True
    use_std=True,
    use_max=True,
    use_min=True
)

# 2. Almacenar resultados
site_channel_maps = {}

for site_name, site_data in tqdm(all_data.items(), desc="Procesando sitios"):
    site_channels = []

    for ts in site_data["data"]:
        # A. Augmentación
        ts_aug = augmentor.augment(ts.copy())

        # B. FC dinámica (W, N, N)
        fcs = fc_builder.compute_dynamic_fc(ts_aug)

        # C. Canales (C, N, N)
        chans = channel_builder.build(fcs)

        site_channels.append(chans)
        
    site_channel_maps[site_name] = site_channels

    print(f"📍 Sitio: {site_name}")
    print(f"   → Sujetos procesados: {len(site_channels)}")
    if len(site_channels) > 0:
        print(f"   → Forma de los canales: {site_channels[0].shape}")
        print(f"   → Tipo: {type(site_channels[0])}")
        print(f"   → Rango de valores: min={site_channels[0].min():.3f}, max={site_channels[0].max():.3f}, mean={site_channels[0].mean():.3f}\n")


NameError: name 'BoldAugmentor' is not defined